#### Prospective Planting

In [ ]:
%load_ext autoreload
%autoreload 2

import sys;
# sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\\')

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [ ]:
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import os
import pandas as pd
import rasterio

from rasterio.mask import mask

import cdl as cd

In [ ]:
def show_excel_index(df,file='check', index=True):
    program =r'"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE"'
    df.to_csv(file+'.csv', index=index,)    
    os.system("start " +program+ " "+file+'.csv')

##### Open USDA CDL

In [ ]:
src=cd.open_cdl(2014)
cdl=src.read(1)

print(cdl.shape)
print(type(cdl))

##### Open the States Shapefile

In [ ]:
usa_states = cd.get_geometry(country='usa', aggregate_level='state')
print(usa_states.shape)
# usa_states

##### Clipping States

In [ ]:
cd.clip_raster_by_shapefile(src, usa_states)

##### Read the Clipped Raster

In [ ]:
state_path = r'\\ac-geneva-24\E\grains trading\Streamlit\LocalData\results\illinois_2008.tif'
state_src=cd.open_raster(state_path)
state_cdl=state_src.read(1)

In [ ]:
cmap = cd.get_cmap(state_src)
cd.plot_raster(state_cdl, cmap)

In [ ]:
values, occurences = np.unique(state_cdl, return_counts=True)

##### Read State Raster Time Series

In [124]:
folder = r'\\ac-geneva-24\E\grains trading\Streamlit\LocalData\results\\'
state = 'illinois'
years=range(2008,2023)
nps={}

for y in years:
    state_path = folder+state+'_'+str(y)+'.tif'
    state_cdl=cd.open_raster(state_path).read(1)
    nps[y]=state_cdl

nps.keys()

dict_keys([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

##### np Array Manipulation  
https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html

In [128]:
# Stacking the rasters so that the dimensions are (lat, lon, year)
crop_matrix=np.stack(nps.values(),axis=2)
crop_matrix.shape

(21061, 11799, 15)

In [ ]:
# Testing masking by plotting only corn pixels
mask=(crop_matrix==1)
crop_matrix[~mask]=0
cd.plot_raster(crop_matrix[:,:,0],cmap,5)

In [129]:
for y in range(0,3):
    print('Year:',y, crop_matrix[:,:,y].shape)

lat = 0 # in pixels
lon = 0 # in pixels
print(f'Crops time series for pixel: {lat},{lon} ----->',y, crop_matrix[0,0,:])

Year: 0 (21061, 11799)
Year: 1 (21061, 11799)
Year: 2 (21061, 11799)
Crops time series for pixel: 0,0 -----> 2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [142]:
year_0=crop_matrix[:,:,0]
year_1=crop_matrix[:,:,1]

df_dict={'before': year_0.ravel(), 'after': year_1.ravel()}

df=pd.DataFrame(df_dict)
df['value']=1

In [159]:
legend = cd.get_legend()
pivot=pd.pivot_table(df, index='before',columns='after', values='value',aggfunc=np.sum)

rename_cols=dict(zip(legend.index, legend['class']))
pivot = pivot.rename(columns=rename_cols)
pivot.index = [rename_cols[i] for i in pivot.index]

probability_df=(pivot.T / pivot.sum(axis=1)).T
show_excel_index(probability_df)

after,Background,Corn,Cotton,Rice,Sorghum,Soybeans,Sunflower,Tobacco,Sweet Corn,Pop or Orn Corn,...,Woody Wetlands,Herbaceous Wetlands,Cantaloupes,Squash,Dbl Crop WinWht/Corn,Pumpkins,Dbl Crop WinWht/Sorghum,Dbl Crop Soybeans/Oats,Dbl Crop Corn/Soybeans,Cabbage
Background,86260638.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Corn,NaN,19488451.0,NaN,134.0,961.0,29345862.0,136.0,4.0,8340.0,19350.0,...,2561.0,18878.0,4.0,41.0,1870.0,14830.0,136.0,43.0,11.0,308.0
Rice,NaN,153.0,NaN,49.0,NaN,308.0,NaN,NaN,NaN,NaN,...,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sorghum,NaN,10144.0,NaN,NaN,232.0,121997.0,NaN,NaN,NaN,NaN,...,6.0,69.0,NaN,1.0,2.0,3.0,2.0,NaN,NaN,NaN
Soybeans,NaN,27187466.0,1.0,8.0,9802.0,5316958.0,269.0,79.0,6272.0,43961.0,...,1837.0,4787.0,35.0,44.0,613.0,2118.0,5488.0,25.0,23.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dbl Crop Corn/Soybeans,NaN,52.0,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabbage,NaN,76.0,NaN,NaN,NaN,13.0,NaN,NaN,84.0,2.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
Turnips,NaN,5.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gourds,NaN,26.0,NaN,NaN,NaN,159.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
